# PREPROCESSING

This notebook contains the process to compile datasets for modeling purpose.

#### The main steps of the process:

- Reading in stock price datasets, resturcturing and creating price based labels

- Reading in article headlines, stemming and tokenizing the text

- Merging and concatenate the article headlines and lables

- Split into train and test dataset

- Saving into a csv file


In order to compare the company's performance I used <b>S&P500</b> index.

## Importing the necessary libraries

In [111]:
# import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot
%matplotlib inline

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_row', 1000)

## 1. Preprocessing the stock price datasets

### 1.1. Importing the dataset 

#### Source: https://finance.yahoo.com

In [406]:
companies = ['tesla', 'ford', 'ibm', 'goldman', 'boeing', 'ge']

In [108]:
sp500 = pd.read_csv('financial/SP500.csv')

tesla_stock = pd.read_csv('financial/TSLA.csv')
ford_stock = pd.read_csv('financial/F.csv')
ibm_stock = pd.read_csv('financial/IBM.csv')
goldman_stock = pd.read_csv('financial/GS.csv')
boeing_stock = pd.read_csv('financial/BA.csv')
ge_stock = pd.read_csv('financial/GE.csv')
jpm_stock = pd.read_csv('financial/JPM.csv')
microsoft_stock = pd.read_csv('financial/MSFT.csv')

### 1.2. Restructuring/labeling stock data

#### Labeling rule:
* <b>good</b>: if the company share price performed better than the index
* <b>bad</b>: if the company share price performed worse than the index

Calculation:
* For calculating changes in stock prices 'Close' prices are used
* 'change': % change compared to the previous day
* 'nextday': % change of the following day

In [289]:
def price_ratios(stock_data, index_data):
    '''
    This function returns a restructured dataframe for labeling purpose.

    The function requires specific structure of the data, which is based on the current 
    datascource (yahoo/finance).

    -------------------------
    Inputs:
        stock_data: dataframe
        index_data: dataframe

    -------------------------
    Returns: dataframe

    '''
    # merge dataframes
    df = pd.merge(stock_data, index_data, how='inner',
                  on='Date', suffixes=('', '_i'))
    
    df.columns = df.columns.str.lower()
    df = df[['date','close', 'close_i', 'volume']]
    
    # shifting prices by one day ahead and merge
    df = df.set_index('date')
    shifted_next = df.shift(periods=-1)
    df = pd.merge(df, shifted_next, on='date', suffixes=('', '_next')) #next day prices
    df = df.reset_index()
     
    # shifting prices by one day back and merge
    df = df.set_index('date')
    shifted_prev = df.shift(periods=1)
    df = pd.merge(df, shifted_prev, on='date', suffixes=('', '_prev')) #previous day prices
    df = df.reset_index()

    # calculate rolling means for the volume
    df['volume_avg'] = df.volume.rolling(window=14).mean().fillna(method='backfill')
    df['volume_ratio'] = round(df.volume / df.volume_avg, 2)
    
    # creating ratios
    df['change'] = round(df['close'] / df['close_prev'] * 100 - 100, 4)
    df['nextday'] = round(df['close_next'] / df['close'] * 100 - 100, 4)
    df['change_i'] = round(df['close_i'] / df['close_i_prev'] * 100 - 100, 4)
    df['nextday_i'] = round(df['close_i_next'] / df['close_i'] * 100 - 100, 4)

    # adding trading holidays to the dataframe and fill
    dates = []
    last_day = pd.to_datetime(stock_data.iloc[-1, 0], format='%Y-%m-%d')
    for i in range(3615):
        date = last_day - datetime.timedelta(days=i)
        date = date.strftime('%Y-%m-%d')
        dates.append(date)
    date_df = pd.DataFrame(dates, columns=['date'])
    date_df = date_df.sort_values(by='date')
    df = pd.merge(date_df, df, on='date', how='left')

    # fill in trading holidays with the next valid data
    df = df.fillna(method='backfill')
    
    df = df[['date', 
             'change', 'nextday', 'change_i', 'nextday_i', 
             'volume_ratio']]
    
    return df

In [256]:
# microsoft_test = price_ratios(microsoft_stock, sp500)

In [257]:
# microsoft_test.date

In [5]:
def label(df):
    df.loc[df['change'] > df['change_i'] + 0.5, 'label'] = 2

    df.loc[df['change'] < df['change_i'] - 0.5, 'label'] = 3

    df.loc[df.label.isna(), 'label'] = 1
    
    df = df[['date', 'label']]
    
#     df.date = pd.to_datetime(df.date)
    
    return df

In [6]:
def label(df):
    df.loc[(df['change'] > df['change_i'])&(df['nextday'] > df['nextday_i']), 'label'] = 1

    df.loc[(df['change'] < df['change_i'])&(df['nextday'] < df['nextday_i']), 'label'] = 0

    df.loc[df.label.isna(), 'label'] = 999
    
    df = df[['date', 'label']]
    
#     df.date = pd.to_datetime(df.date)
    
    return df

In [290]:
# this labeling takes into consideration the volume and the relative change

def label(df):
    df.loc[(df['change'] > df['change_i'] + 0.5)&(df['volume_ratio'] > 1), 'label'] = 1

    df.loc[(df['change'] < df['change_i'] - 0.5)&(df['volume_ratio'] > 1), 'label'] = 0

    df.loc[df.label.isna(), 'label'] = 999
    
#     df = df[['date', 'label']]
    
#     df.date = pd.to_datetime(df.date)
    
    return df

In [291]:
tesla_label = label(price_ratios(tesla_stock, sp500))
ford_label = label(price_ratios(ford_stock, sp500))
ibm_label = label(price_ratios(ibm_stock, sp500))
goldman_label = label(price_ratios(goldman_stock, sp500))
boeing_label = label(price_ratios(boeing_stock, sp500))
ge_label = label(price_ratios(ge_stock, sp500))
jpm_label = label(price_ratios(jpm_stock, sp500))
microsoft_label = label(price_ratios(microsoft_stock, sp500))

In [269]:
labels = {1: 'good', 999: 'neutral', 0: 'bad'}
boeing_label.label.value_counts()

999.0    2647
0.0      493 
1.0      475 
Name: label, dtype: int64

## 2. Preprocessing the article information

### 2.1. Import headlines

In [207]:
import pymongo

myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
news = myclient['news']

In [293]:
tesla_news = pd.DataFrame.from_records(news.tesla_news.find())
ford_news = pd.DataFrame.from_records(news.ford_news.find())
ibm_news = pd.DataFrame.from_records(news.ibm_news.find())
goldman_news = pd.DataFrame.from_records(news.goldman_news.find())
boeing_news = pd.DataFrame.from_records(news.boeing_news.find())
ge_news = pd.DataFrame.from_records(news.ge_news.find())
jpmorgan_news = pd.DataFrame.from_records(news.jpmorgan_news.find())
microsoft_news = pd.DataFrame.from_records(news.microsoft_news.find())

### 2.2. Concatenate headline and summary into one text

In [298]:
def concat_news(some_news):
    some_news['text'] = some_news.headline +' '+ some_news.summary
    some_news = some_news[['date', 'text', 'topic']]
    return some_news

In [299]:
tesla_news = concat_news(tesla_news)
ford_news = concat_news(ford_news)
ibm_news = concat_news(ibm_news)
goldman_news = concat_news(goldman_news)
boeing_news = concat_news(boeing_news)
ge_news = concat_news(ge_news)
jpmorgan_news = concat_news(jpmorgan_news)
microsoft_news = concat_news(microsoft_news)

### 2.3. Keeping just the relevant articles

Relevant a headline or the summary, if it contains the keywords (basically the company name).

In [300]:
def relevant(text, keywords):
    '''
    Inputs:
        text: string
        keywords: list of string objects
    Returns:
        boolean
    '''
    for keyword in keywords:
        if keyword in text:
            return True
    return False

def relevant_rows(df, keywords):
    '''
    Returns the rows of the dataframe, where the text is relevant
    Input:
        df: dataframe
        keywords: list of string objects
    Returns:
        dataframe with the relevant rows
    '''
    df = df.loc[df.text.apply(lambda x: relevant(x, keywords))]
    
    print('Number of rows: ',len(df))
    return df

In [301]:
tesla_news = relevant_rows(tesla_news, ['Tesla'])

Number of rows:  933


In [302]:
ford_news = relevant_rows(ford_news, ['Ford'])

Number of rows:  1447


In [303]:
ibm_news = relevant_rows(ibm_news, ['IBM', 'International Business Machines'])

Number of rows:  509


In [304]:
goldman_news = relevant_rows(goldman_news, ['Goldman'])

Number of rows:  1667


In [305]:
boeing_news = relevant_rows(boeing_news, ['Boeing'])

Number of rows:  1629


In [306]:
ge_news = relevant_rows(ge_news, ['GE', 'General Electric'])

Number of rows:  1085


In [307]:
jpmorgan_news = relevant_rows(jpmorgan_news, ['JPMorgan'])

Number of rows:  223


In [308]:
microsoft_news = relevant_rows(microsoft_news, ['Microsoft'])

Number of rows:  1416


### 2.4. Converting dates 

If the article is published after the closing time of the stock market, I convert its date into the next day, becaues this article can affect only the next day price. 

In [310]:
def converting_date(some_date):
    # compares the published time of the article with the closure of the stockmarket
    # if the article is published after the closure, it's date changed to the next day
    if pd.to_datetime(some_date).time() > datetime.time(16, 0):
        day = pd.to_datetime(some_date).date() + datetime.timedelta(days=1)
    else:
        day = pd.to_datetime(some_date).date()
    return day.strftime('%Y-%m-%d')

In [311]:
tesla_news.date = tesla_news.date.apply(converting_date)
ford_news.date = ford_news.date.apply(converting_date)
ibm_news.date = ibm_news.date.apply(converting_date)
microsoft_news.date = microsoft_news.date.apply(converting_date)
goldman_news.date = goldman_news.date.apply(converting_date)
jpmorgan_news.date = jpmorgan_news.date.apply(converting_date)
boeing_news.date = boeing_news.date.apply(converting_date)
ge_news.date = ge_news.date.apply(converting_date)

### 2.5. Stemming/tokenizing

The 'stopwords' and 'negativewords' were originally based on the stopwords of nltk library. For this sentiment
analysis I reviewed and modified the content of stopwords (removed keywords, added some not predicting words).
Words that change completely the meaning of the following words (I call them like 'negative words') will be replaced simply with 'not'.

In [312]:
my_stopwords = ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an',
                'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being',
                'between', 'both', 'but', 'by', 'can', 'd', 'did', 'do', 'does', 'doing', 
                'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 
                'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 
                'how', 'i', 'if', 'in', 'into', 'is', 'it', "it's", 'its', 'itself', 'just', 
                'll', 'm', 'ma', 'me', 'might', 'more', 'most', 'my', 'myself', 'neeed', 
                'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 
                'ourselves', 'out', 'over', 'own', 're', 's', 'u', 't', 'same', 'she', 
                "she's", 'should', "should've", 'so', 'some', 'such', 'than', 'that', "that'll",
                'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 
                'they', 'this', 'k', 'those', 'through', 'to', 'too', 'until', 've', 'very', 
                'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 
                'why', 'will', 'with', 'would', 'could', 'y', 'you', "you'd", "you'll", 
                "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 'mr', 'mrs', 
                'ms', 'nan', 'inc', 'co', 'com', 'wsj', 'monday', 'tuesday', 'wednesday', 
                'thursday', 'friday', 'saturday', 'sunday', 'either', 'shall', 'must', 'with', 
                'without', 'may']

# my_negativewords = ["aren't", "couldn't", "didn't", "doesn't", "don't",
#                     "hadn't", "hasn't", "haven't", "isn't", "mightn't",
#                     'neither', 'nor', "mustn't", "needn't", 'no', "shan't",
#                     "shouldn't", "wasn't", "weren't", "won't", "wouldn't"]


my_negativewords = ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 
                    'mightn', 'no', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 
                    'won', 'wouldn', 'neither', 'nor']

In [313]:
import re
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()


def my_tokenizer(text):
    # just keep the words (remove characters and numbers)
    token_list = re.findall(r'([a-zA-Z-]+)', text)

    # lowercase
    token_list = [t.lower() for t in token_list]

    # remove stopwords
    token_list = [word for word in token_list if word not in my_stopwords]

    # replace negative words to 'not'
    for i, word in enumerate(token_list):
        if word in my_negativewords:
            token_list[i] = 'not'
    
    # lemmatize
    token_list = [wnl.lemmatize(word) for word in token_list]
    
    # converting list into text
    token_text = ' '.join(token_list)

    return token_text

In [314]:
tesla_news['tokens'] = tesla_news.text.apply(my_tokenizer)
ford_news['tokens'] = ford_news.text.apply(my_tokenizer)
ibm_news['tokens'] = ibm_news.text.apply(my_tokenizer)
goldman_news['tokens'] = goldman_news.text.apply(my_tokenizer)
boeing_news['tokens'] = boeing_news.text.apply(my_tokenizer)
ge_news['tokens'] = ge_news.text.apply(my_tokenizer)
jpmorgan_news['tokens'] = jpmorgan_news.text.apply(my_tokenizer)
microsoft_news['tokens'] = microsoft_news.text.apply(my_tokenizer)

## 3. Merging/concatenating

In [315]:
tesla = pd.merge(tesla_news, tesla_label, how='left', on='date')
ford = pd.merge(ford_news, ford_label, how='left', on='date')
ibm = pd.merge(ibm_news, ibm_label, how='left', on='date')
goldman = pd.merge(goldman_news, goldman_label, how='left', on='date')
boeing = pd.merge(boeing_news, boeing_label, how='left', on='date')
ge = pd.merge(ge_news, ge_label, how='left', on='date')
jpm = pd.merge(jpmorgan_news, jpm_label, how='left', on='date')
microsoft = pd.merge(microsoft_news, microsoft_label, how='left', on='date')

In [321]:
data = pd.concat([tesla, ford, ibm, goldman, boeing, ge, jpm, microsoft])

#### Exploration on topics

In [372]:
# data.loc[data.label!=999].topic.value_counts().index.to_list()[:26]

In [371]:
# data.loc[data.topic=='REVIEW & OUTLOOK (U.S.)']

In [373]:
# data.loc[data.date=='2011-01-07']

### 3.1. Removing not relevant topics

In [356]:
# first I determine the relevant topics
relevant_topics = data.loc[data.label!=999].topic.value_counts().index.to_list()[:26]
for elem in ['PHOTOS', 'LETTERS', 'COMMENTARY', 'MANAGEMENT', 'POLITICS', 'BUSINESS WORLD',
             'MULTIMEDIA', 'REVIEW & OUTLOOK (U.S.)']:
    relevant_topics.remove(elem)
    
def relevant_topic(topic):
    if topic in relevant_topics:
        return True
    return False

data = data.loc[data.topic.apply(lambda x: relevant_topic(x))]

### 3.2. Removing not polarizing articles

In [357]:
data.label.value_counts()

999.0    4417
0.0      1348
1.0      1305
Name: label, dtype: int64

In [358]:
data = data.loc[data.label != 999]

In [359]:
data.shape

(2653, 10)

In [363]:
data = data.reset_index()


In [364]:
data.drop(columns = 'index', inplace=True)

In [365]:
data.tail(2)

,date,text,topic,tokens,change,nextday,change_i,nextday_i,volume_ratio,label
2651,2010-01-15,"Microsoft, Abbott Laboratories: Money Flow Leaders (MSFT, ABT) Markets Data Center: Money Flow Leaders.",MARKETS,microsoft abbott laboratory money flow leader msft abt market data center money flow leader,-0.323,0.7777,-1.0823,1.25,1.31,1.0
2652,2010-01-15,"Intel, IBM Lift Dow; Baidu Hits New Peak Stocks rose as hopes for a strong earnings report from Intel boosted technology stocks including Microsoft and International Business Machines.",nan,intel ibm lift dow baidu hit new peak stock rose hope strong earnings report intel boosted technology stock including microsoft international business machine,-0.323,0.7777,-1.0823,1.25,1.31,1.0


## 4. Train-test split

In [366]:
data = data.sort_values(by='date', ascending=False)

In [367]:
data_test = data[:350]
data_train = data[350:]

In [93]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data,
                                         test_size=.20,
                                         random_state=10,
                                         stratify=data.label)

In [368]:
data_train.label.value_counts()

0.0    1158
1.0    1145
Name: label, dtype: int64

In [369]:
data_test.label.value_counts()

0.0    190
1.0    160
Name: label, dtype: int64

## 5. Save into a csv file

In [370]:
data_train.to_csv('train.csv')
data_test.to_csv('test.csv')